In [1]:
# 라이브러리 불러오기 및 함수화
import re
import pandas as pd

from konlpy.tag import Komoran
from pykospacing import Spacing
from sklearn.feature_extraction.text import CountVectorizer

komoran = Komoran()
spacing = Spacing()

In [4]:
df = pd.read_csv(r'..\..\..\data\combined_questions.csv', encoding='cp949')
# df = df[:][0:100]

# 불용어 리스트 가져오기
def get_stopwords():
    stopwords = set()
    with open('../../../data/stopwords-ko.txt', 'r', encoding='utf-8') as file:
        for line in file:
            stopwords.add(line.strip())
    return stopwords

# 문장 전처리 함수
def preprocessing(text):
    text = spacing(text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# 형태소 분석 및 불용어 제거 함수
def morph_and_remove_stopwords(text, stopwords):
    text = preprocessing(text)
    morphs = komoran.morphs(text)
    meaningful_words = [word for word in morphs if word not in stopwords]
    return meaningful_words

In [5]:
# 불용어 리스트 가져오기
stopwords = get_stopwords()

# 데이터프레임의 'feature' 열에 대해 형태소 분석 및 불용어 제거 적용
df['processed_feature'] = df['feature'].apply(lambda x: morph_and_remove_stopwords(x, stopwords))

# processed_feature를 문자열로 변환
df['processed_feature_str'] = df['processed_feature'].apply(lambda x: ' '.join(x))

# CountVectorizer를 사용하여 BoW 벡터화
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['processed_feature_str'])

# BoW 결과를 데이터프레임으로 변환
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# 원래 데이터프레임과 BoW 데이터프레임을 병합
df_bow = pd.concat([df, bow_df], axis=1)